In [32]:
import requests
import json
import hashlib
import os
from datetime import datetime as datingdays
from git import Repo, Git
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
from commit_log_parser import StatRequirementSet
from pytz import timezone

def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))
        
def loadCachedURL(url, forceReload = False):
    body = None
    hl = hashlib.new('sha256')
    ba = bytearray(url.encode())
    hl.update(ba)
    thing = hl.hexdigest()
    cachedFileName = './'+thing
    loaded = False;
    if not forceReload:
        try:
            with open(cachedFileName, 'r') as f:
                body = json.load(f)
                f.close()
            loaded = True
        except EnvironmentError:
            pass

    if not loaded:
        resp = requests.get(url)
        if (resp.status_code == 200):
            body = resp.json()
            with open(cachedFileName, 'w') as f:
                f.write(resp.text)
                f.close()
    return body
class Commit:
    sha = None
    date = None
    hacker = None
class Alias:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    def __hash__(self):
        return hash((self.name, self.email))

    def __eq__(self, other):
        return (self.name, self.email) == (other.name, other.email)

    def __ne__(self, other):
        return not(self == other)        
class Hacker:
    def __init__(self):
        self.user_id = None
        self.commits = []
        self.aliases = Counter()
    def addAlias(self, alias):
        aliases[alias] += 1
    def __init__(self, user_id):
        self.user_id = user_id;
class Repository:
    def __init__(self, repo_id, repo_name):
        self.repo_id = repo_id
        self.repo_name = repo_name
# Types of queries:
#
#  Single query to derive a user ID
#  Mass commit log
class Query:
    def __init__(self):
        self.urlPrefix = 'https://api.github.com/search/commits'
        self.startDate = datingdays.now(timezone('US/Arizona'))
        self.hacker = None
        self.repos = {}
        with open('./web3.github.token', 'r') as f:
            self.token = f.readline()
            self.token = self.token.strip('\n')
            self.headers = {'Authorization': 'token %s' % self.token}
    def set_last_date(self, date):
        new_date = datingdays.fromisoformat(date)
        if (new_date < self.startDate):
            self.startDate = self.startDate - (self.startDate - new_date)
    def format_url(self):
        var = self.urlPrefix+"?q=author:"+self.hacker.user_id+'+author-date:<'+self.startDate.isoformat()+'&sort=author-date&order=desc&per_page=100&page=1'
        print(var)
        return var
    def load_url(self):
        retVal = None
        resp = requests.get(self.format_url(), headers=self.headers)
        if (resp.status_code == 200):
            retVal = resp.json()
        return retVal
    
    
owner = 'Deadman-DAO'
repo_name = 'Web3HackerNetwork'
repo_base_dir = './repos'
repo_path = repo_base_dir+'/'+repo_name

if (os.path.isdir(repo_base_dir) == False):
    print('######### ERROR - Cannot find '+repo_base_dir+'  Creating it!')
    os.path.mkdir(repo_base_dir)
url = 'https://github.com/'+owner+'/'+repo_name+'.git'
if (os.path.isdir(repo_path) == False):
    Repo.clone_from(url, repo_path)
else:
    rp = Repo(repo_path)
    remote = rp.remote()
    remote.pull()
        
rep = Git(repo_path)
stat = rep.log('--stat')

stat_req_set = StatRequirementSet()
stat_req_set.processDocument(stat)
#print(json.dumps(stat_req_set.resultArray, indent=2))

#print(rep.log('--numstat'))
    
q = Query()
q.hacker = Hacker('marhali')    
done = False
while not done:
    body = q.load_url()
     
    if (body == None):
        print('Unable to load JSON')
        done = True
    else:
        total_count = body['total_count']
        incomplete_results = body['incomplete_results']
        print(total_count)
        print(incomplete_results)
        array = body['items'];
        if (array == None or len(array) < 1):
            done = True
        else:
            for n in array:
                author = n['author']
                login = author['login']
                repo = n['repository']
                repoId = repo['id']
                repoName = repo['name']
                commit = n['commit']
                sha = n['sha']
                comAuth = commit['author']
                authorName = comAuth['name']
                authorDate = comAuth['date']
                q.set_last_date(authorDate)
                print(login, repoId, repoName, authorName, authorDate, sha)
        


https://api.github.com/search/commits?q=author:marhali+author-date:<2022-05-04T13:00:28.033971-07:00&sort=author-date&order=desc&per_page=100&page=1
1525
False
marhali 344076009 easy-i18n Marcel 2022-04-28T18:20:41.000+02:00 8e6f7098afa53957a8bd514c50a35c9072c9305c
marhali 344076009 easy-i18n marhali 2022-04-26T20:58:41.000+02:00 f2df66238bad95828545b4902707c4a58d55a03c
marhali 348131001 ocean-pearl-api Marcel 2022-04-24T15:24:52.000+02:00 662147c673d76e750cf187fb2e33965f29b22bdd
marhali 344076009 easy-i18n marhali 2022-04-24T15:23:17.000+02:00 5abd9c3dbfb500cc546a665b0b73bd07d9d2fbe1
marhali 344076009 easy-i18n marhali 2022-04-24T15:22:40.000+02:00 14a41ab6a4a4673c731777d4677ceefce225715f
marhali 348131001 ocean-pearl-api Marcel 2022-04-24T15:20:12.000+02:00 94544ecbc28c4f3dd9d6db1ea82b36d2ef59ac2e
marhali 348131001 ocean-pearl-api Marcel 2022-04-24T13:02:28.000+02:00 27299f42582506e9799ffa8b961ba131168c1742
marhali 348131001 ocean-pearl-api Marcel 2022-04-24T13:00:40.000+02:00 2a3c7d

In [ ]:
from datetime import datetime as datingdays
fmt = '%Y-%m-%dT%H:%M:%S.%f%z'
t = '2020-01-28T15:47:53.000+01:00'
rslt = datingdays.strptime(t, fmt)
print(rslt)
print(rslt.strftime(fmt))


In [ ]:
import urllib.parse
from github import Github
import os
xxx = urllib.parse.parse_qs('https://api.github.com/search/commits?q=author:marhali+author-date:<2022-06-01&sort=author-date&order=desc&per_page=100&page=1')
print(xxx)

In [26]:
from datetime import datetime as dt
from pytz import timezone
d = dt.fromisoformat('2022-04-11T19:14:33.000+02:00')

#az = timezone('US/Arizona')
#d2 = az.localize(d, is_dst=False)
#d2              


type(obj):		<class 'datetime.timezone'>
id(obj):		140218538711504
dir(obj):		['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getinitargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'dst', 'fromutc', 'max', 'min', 'tzname', 'utc', 'utcoffset']


TypeError: vars() argument must have __dict__ attribute

In [27]:
from datetime import datetime
from datetime import timedelta
# Given timestamp in string
time_str = '24/7/2021 11:13:08.230010'
date_format_str = '%d/%m/%Y %H:%M:%S.%f'
# create datetime object from timestamp string
given_time = datetime.strptime(time_str, date_format_str)
print('Given Time: ', given_time)
n = 2
# Subtract 2 hours from datetime object
final_time = given_time - timedelta(hours=n)
print('Final Time (2 hours ahead of given time ): ', final_time)
# Convert datetime object to string in specific format 
final_time_str = final_time.strftime('%d/%m/%Y %H:%M:%S.%f')
print('Final Time as string object: ', final_time_str)

Given Time:  2021-07-24 11:13:08.230010
Final Time (2 hours ahead of given time ):  2021-07-24 09:13:08.230010
Final Time as string object:  24/07/2021 09:13:08.230010
